In [1]:
#FOr the 2nd task that is predicting gender and age through videos and webcam and identify whether the person in the video is a senior citizen or not.
#Now to train a model 
#Steps from kaggle we will first collect images, load in this jupyter notebook, then through these images we will train the model to predict the age and gender then directly on the videos


In [3]:
#Now for images first i have downloaded a zip file from UTKFace from kaggle i will upload these file(images) to train the model for videos

In [5]:
#Now first lets import opencv
import cv2

#Now as the task is for images and videos which are prerecorded 
#1. for images  
cv2.imread("") 
#2. Will load videos 
cv2.VideoCapture("")



import os, cv2, numpy as np                            #cv2= Opencv which a CNN
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical



path= r"C:\Users\HP\Downloads\archive\UTKFace"  #Extracted file's path as it was a zip file before, so had to



X, ages, genders = [], [], []
for file in os.listdir(path):
    if file.endswith(".jpg"):
        try:
            parts = file.split("_") 
            age = int(parts[0])
            gender = int(parts[1])   
            img = cv2.imread(os.path.join(path, file))
            img = cv2.resize(img, (64,64)) / 255.0
            X.append(img)
            ages.append(age)
            genders.append(gender)
        except:
            continue

print("Loaded:", len(X), "images")




Loaded: 23708 images


In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y_age = np.array(ages)
y_gender = to_categorical(genders, 2)

X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42
)


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

inp = Input(shape=(64,64,3))
x = Conv2D(32,(3,3),activation='relu')(inp); x = MaxPooling2D()(x)
x = Conv2D(64,(3,3),activation='relu')(x); x = MaxPooling2D()(x)
flat = Flatten()(x)

age_out = Dense(1,activation='linear',name='age')(flat)      
gen_out = Dense(2,activation='softmax',name='gender')(flat)   

model = Model(inp,[age_out,gen_out])
model.compile(optimizer='adam',
              loss={'age':'mse','gender':'categorical_crossentropy'},
              metrics={'age':'mae','gender':'accuracy'})


In [11]:
model.fit(X_train,[y_age_train,y_gender_train],
          validation_data=(X_test,[y_age_test,y_gender_test]),
          epochs=5,batch_size=32)


Epoch 1/5
593/593 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - age_loss: 352.0059 - age_mae: 14.2528 - gender_accuracy: 0.7278 - gender_loss: 0.5594 - loss: 352.5909 - val_age_loss: 218.3186 - val_age_mae: 11.4936 - val_gender_accuracy: 0.7632 - val_gender_loss: 0.5490 - val_loss: 219.2318
Epoch 2/5
593/593 ━━━━━━━━━━━━━━━━━━━━ 34s 58ms/step - age_loss: 223.9147 - age_mae: 11.4812 - gender_accuracy: 0.8152 - gender_loss: 0.4621 - loss: 224.3577 - val_age_loss: 196.9024 - val_age_mae: 10.5701 - val_gender_accuracy: 0.8458 - val_gender_loss: 0.3632 - val_loss: 197.4920
Epoch 3/5
593/593 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - age_loss: 198.2096 - age_mae: 10.7892 - gender_accuracy: 0.8335 - gender_loss: 0.4307 - loss: 198.6715 - val_age_loss: 187.2475 - val_age_mae: 10.3836 - val_gender_accuracy: 0.8277 - val_gender_loss: 0.4496 - val_loss: 187.9192
Epoch 4/5
593/593 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - age_loss: 176.1324 - age_mae: 10.1624 - gender_accuracy: 0.8454 - gender_loss: 0.4073 - loss: 17

In [13]:
model.save("age_gender_seniorcitizen_model.h5")



In [15]:
model.save("age_gender_seniorcitizen_model.keras")


In [ ]:
#To predict 2-3 people's age gender and whether they are citizen or not from a single video we will these codes 

In [90]:
import cv2

video_path = r"C:\Users\HP\Videos\test.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(" Could not open video.")
    exit()

print("Video opened ")
print("Frame width:", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Frame height:", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Total frames:", cap.get(cv2.CAP_PROP_FRAME_COUNT))

while True:
    ret, frame = cap.read()
    if not ret:
        print(" No more frames or cannot read frame.")
        break

    cv2.imshow("Frame", frame)   # just display raw video
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


❌ Could not open video.
Video opened ✅
Frame width: 0.0
Frame height: 0.0
Total frames: 0.0
⚠️ No more frames or cannot read frame.


In [33]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from datetime import datetime



model = load_model("age_gender_seniorcitizen_model.keras")



cap = cv2.VideoCapture(r"C:\Users\HP\Downloads\5270503-uhd_3840_2160_25fps.mp4")
if not cap.isOpened():
    print("Error: Cannot open video source")
    exit()

log = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("No more frames or video cannot be read.")
        break

    
    face = cv2.resize(frame, (64,64)) / 255.0
    face = np.expand_dims(face, axis=0)

    
    pred_age, pred_gender = model.predict(face)
    age = int(pred_age[0][0])
    gender = "Male" if np.argmax(pred_gender) == 1 else "Female"
    senior = "Yes" if age >= 60 else "No"

    
    cv2.putText(frame, f"Age:{age} Gender:{gender} Senior:{senior}",
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0,255,0),2)

    
    cv2.imshow("Video", frame)

    
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log.append([now, age, gender, senior])

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



cap.release()
cv2.destroyAllWindows()




df = pd.DataFrame(log, columns=["DateTime","Age","Gender","SeniorCitizen"])
df.to_csv("video_log.csv", index=False)
print("Log saved ")



C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━

In [35]:
from tensorflow.keras.models import load_model

model = load_model("age_gender_seniorcitizen_model.keras")


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [37]:
import cv2


video_path = r"C:\Users\HP\Downloads\5270503-uhd_3840_2160_25fps.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video")


In [39]:
import numpy as np
from datetime import datetime
import pandas as pd

log = []

while True:
    ret, frame = cap.read()
    if not ret:
        print(" No more frames or video cannot be read.")
        break

    
    face = cv2.resize(frame, (64, 64)) / 255.0
    face = np.expand_dims(face, axis=0) 

    
    pred_age, pred_gender = model.predict(face)
    age = int(pred_age[0][0])
    gender = "Male" if np.argmax(pred_gender) == 1 else "Female"
    senior = "Yes" if age >= 60 else "No"


    
    cv2.putText(frame, f"Age:{age} Gender:{gender} Senior:{senior}",
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)



    
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log.append([now, age, gender, senior])



    
    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━

In [41]:
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 10 != 0: 
        continue


In [43]:
all_ages = []
all_genders = []



all_ages.append(age)
all_genders.append(gender)


final_age = int(np.mean(all_ages))
final_gender = max(set(all_genders), key=all_genders.count)
senior = "Yes" if final_age >= 60 else "No"
print(final_age, final_gender, senior)


49 Female No


In [49]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.models import load_model



model = load_model("age_gender_seniorcitizen_model.keras")

# For webcam use 0, i have not used for safety
cap = cv2.VideoCapture(r"C:\Users\HP\Downloads\5270503-uhd_3840_2160_25fps.mp4")

if not cap.isOpened():
    print("Error: Cannot open video source")
    exit()

all_ages = []
all_genders = []

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    
    if frame_count % 10 != 0:
        continue

    
    face = cv2.resize(frame, (64, 64)) / 255.0
    face = np.expand_dims(face, axis=0)

    
    pred_age, pred_gender = model.predict(face, verbose=0)
    age = int(pred_age[0][0])
    gender = "Male" if np.argmax(pred_gender) == 1 else "Female"

    all_ages.append(age)
    all_genders.append(gender)

cap.release()


final_age = int(np.mean(all_ages))
final_gender = max(set(all_genders), key=all_genders.count)
senior = "Yes" if final_age >= 60 else "No"


now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
df = pd.DataFrame([[now, final_age, final_gender, senior]],
                  columns=["DateTime", "Age", "Gender", "SeniorCitizen"])
df.to_csv("video_log_new.csv", index=False)
print("Prediction saved ")
print(f"Age: {final_age}, Gender: {final_gender}, Senior: {senior}")


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Prediction saved ✅
Age: 44, Gender: Female, Senior: No


In [ ]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from datetime import datetime


model = load_model("age_gender_seniorcitizen_model.keras")



face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Need to 0 for webcam, same code for video
cap = cv2.VideoCapture(r"C:\Users\HP\Downloads\6063269-uhd_4096_2160_25fps.mp4")


log = []

while True:
    ret, frame = cap.read()
    if not ret:
        print(" No more frames or video cannot be read.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (64, 64)) / 255.0
        face_input = np.expand_dims(face_resized, axis=0)

        pred_age, pred_gender = model.predict(face_input)
        age = int(pred_age[0][0])
        gender = "Male" if np.argmax(pred_gender) == 1 else "Female"
        senior = "Yes" if age >= 60 else "No"

        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, f"{age}, {gender}, Senior:{senior}", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log.append([now, age, gender, senior])

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


df = pd.DataFrame(log, columns=["DateTime", "Age", "Gender", "SeniorCitizen"])
df.to_csv(r"C:\Users\HP\Desktop\video_log_multiple.csv", index=False)
print("Log saved with all detected faces")


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━

In [61]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from datetime import datetime


model = load_model("age_gender_seniorcitizen_model.keras")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(r"C:\Users\HP\Downloads\video_with_3_people.mp4")




people = {}  
face_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (64, 64)) / 255.0
        face_input = np.expand_dims(face_resized, axis=0)

        pred_age, pred_gender = model.predict(face_input)
        age = int(pred_age[0][0])
        gender = "Male" if np.argmax(pred_gender) == 1 else "Female"

        
        if face_id not in people:
            people[face_id] = {'ages': [], 'genders': []}
            face_id += 1

        people[face_id-1]['ages'].append(age)
        people[face_id-1]['genders'].append(gender)

cap.release()
cv2.destroyAllWindows()


log = []
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
for person in people.values():
    final_age = int(np.mean(person['ages']))
    final_gender = max(set(person['genders']), key=person['genders'].count)
    senior = "Yes" if final_age >= 60 else "No"
    log.append([now, final_age, final_gender, senior])


df = pd.DataFrame(log, columns=["DateTime", "Age", "Gender", "SeniorCitizen"])
df.to_csv(r"C:\Users\HP\Desktop\video_log_3people.csv", index=False)
print("Predictions saved for 3 people")


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ Predictions saved for 3 people


In [1]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from datetime import datetime


model = load_model("age_gender_seniorcitizen_model.keras")

cap = cv2.VideoCapture(r"C:\Users\HP\Downloads\video_with_3_people.mp4")



predictions = {}  
person_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    faces = [cv2.resize(frame, (64, 64))]  

    for i, face in enumerate(faces):
        face_input = np.expand_dims(face/255.0, axis=0)
        pred_age, pred_gender = model.predict(face_input)
        age = int(pred_age[0][0])
        gender = "Male" if np.argmax(pred_gender) == 1 else "Female"
        senior = "Yes" if age >= 60 else "No"

        
        if i+1 not in predictions:
            predictions[i+1] = [age, gender, senior]



log = []
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
for person_id, info in predictions.items():
    log.append([now, info[0], info[1], info[2]])

df = pd.DataFrame(log, columns=["DateTime", "Age", "Gender", "SeniorCitizen"])
df.to_csv(r"C:\Users\HP\Desktop\video_log_3people_unique.csv", index=False)
print("One entry per person saved successfully!")

cap.release()
cv2.destroyAllWindows()


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


One entry per person saved successfully!


In [ ]:
#requirement text

#tensorflow
#keras
#opencv-python
#numpy
#matplotlib
#Scikit-learn
#Pillow


In [3]:
import os
print("Working dir:", os.getcwd())
for name in sorted(os.listdir(os.getcwd())):
    if name.lower().endswith((".ipynb",".pt",".pth",".h5",".hdf5",".csv",".xlsx")):
        print(name)


Working dir: C:\Users\HP
1. Long Hair_model.ipynb
1.ipynb
1123.ipynb
2.Gender, senior citizen_model.ipynb
3.Age&emotions_model.ipynb
4.Sign language.ipynb
5.Car detection.ipynb
6.Nationality model.ipynb
Advanced data science.ipynb
Array.ipynb
Data science project.ipynb
Python 35.ipynb
Python project.ipynb
Senior citizen detection.ipynb
Untitled.ipynb
Untitled3.ipynb
age.pt
age_gender_model.h5
age_gender_seniorcitizen_model.h5
age_gender_senirocitizen_model.h5
emotion.pt
emotion_model.h5
hair_model.pt
senior_citizens.csv
senior_results.csv
trial.ipynb
video_log.csv
video_log_new.csv
yolov8n.pt


In [1]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime
from tkinter import Tk, Button, filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model("age_gender_seniorcitizen_model.keras")

AGE_LABELS = ["0-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-90", "91-100"]
GENDER_LABELS = ["Male", "Female"]

log_file = "visit_log.csv"
header = ["age_bucket", "gender", "is_senior", "timestamp"]

if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

def log_entry(age_bucket, gender):
    try:
        min_age = int(age_bucket.split("-")[0])
    except:
        min_age = 0
    is_senior = 1 if min_age >= 60 else 0
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([age_bucket, gender, is_senior, ts])

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def predict_age_gender(face_img):
    face = cv2.resize(face_img, (64, 64))
    face = face.astype("float") / 255.0
    face = img_to_array(face)
    face = np.expand_dims(face, axis=0)
    preds = model.predict(face)
    if isinstance(preds, list) and len(preds) == 2:
        age_probs, gender_probs = preds
        age_bucket = AGE_LABELS[np.argmax(age_probs[0])]
        gender = GENDER_LABELS[np.argmax(gender_probs[0])]
    else:
        probs = preds[0]
        age_bucket = AGE_LABELS[np.argmax(probs[:len(AGE_LABELS)])]
        gender = GENDER_LABELS[np.argmax(probs[len(AGE_LABELS):])]
    return age_bucket, gender

def process_video(cap):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            age_bucket, gender = predict_age_gender(face_img)
            log_entry(age_bucket, gender)
            label = f"{gender}, {age_bucket}"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, label, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("Senior Citizen Identification", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

def open_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    if file_path:
        cap = cv2.VideoCapture(file_path)
        process_video(cap)

def open_webcam():
    cap = cv2.VideoCapture(0)
    process_video(cap)

root = Tk()
root.title("Senior Citizen Identification")

btn_video = Button(root, text="Upload Video", command=open_video)
btn_video.pack(pady=10)

btn_webcam = Button(root, text="Open Webcam", command=open_webcam)
btn_webcam.pack(pady=10)

root.mainloop()


C:\Users\HP\anaconda3\envs\longhairenv\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("age_gender_seniorcitizen_model.keras", compile=False)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
import os
print(os.getcwd())


In [14]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime
from tkinter import Tk, Button, filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model(r"C:\Users\HP\Documents\Nullclass internship Tasks\Task 2 -  Senior citizen Identification\age_gender_seniorcitizen_model.keras")


AGE_LABELS = ["0-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-90", "91-100"]
GENDER_LABELS = ["Male", "Female"]

log_file = "visit_log.csv"
header = ["age_bucket", "gender", "is_senior", "timestamp"]

if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

def log_entry(age_bucket, gender):
    try:
        min_age = int(age_bucket.split("-")[0])
    except:
        min_age = 0
    is_senior = 1 if min_age >= 60 else 0
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([age_bucket, gender, is_senior, ts])
    print(f"[LOGGED] Age: {age_bucket}, Gender: {gender}, Senior: {is_senior}, Time: {ts}")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def predict_age_gender(face_img):
    face = cv2.resize(face_img, (64, 64))
    face = face.astype("float") / 255.0
    face = img_to_array(face)
    face = np.expand_dims(face, axis=0)
    preds = model.predict(face)
    if isinstance(preds, list) and len(preds) == 2:
        age_probs, gender_probs = preds
        age_bucket = AGE_LABELS[np.argmax(age_probs[0])]
        gender = GENDER_LABELS[np.argmax(gender_probs[0])]
    else:
        probs = preds[0]
        age_bucket = AGE_LABELS[np.argmax(probs[:len(AGE_LABELS)])]
        gender = GENDER_LABELS[np.argmax(probs[len(AGE_LABELS):])]
    return age_bucket, gender

def process_video(cap):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            age_bucket, gender = predict_age_gender(face_img)
            log_entry(age_bucket, gender)
            label = f"{gender}, {age_bucket}"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("Senior Citizen Identification", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

def open_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    if file_path:
        cap = cv2.VideoCapture(file_path)
        process_video(cap)

def open_webcam():
    cap = cv2.VideoCapture(0)
    process_video(cap)

root = Tk()
root.title("Senior Citizen Identification")

btn_video = Button(root, text="Upload Video", command=open_video)
btn_video.pack(pady=10)

btn_webcam = Button(root, text="Open Webcam", command=open_webcam)
btn_webcam.pack(pady=10)

root.mainloop()


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

X = np.random.rand(500,64,64,3).astype(np.float32)
y_age = np.random.randint(0,10,500)
y_gender = np.random.randint(0,2,500)

y_age_oh = tf.keras.utils.to_categorical(y_age, num_classes=10)
y_gender_oh = tf.keras.utils.to_categorical(y_gender, num_classes=2)

inp = Input(shape=(64,64,3))
x = Conv2D(32,(3,3),activation="relu")(inp)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64,(3,3),activation="relu")(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(128,activation="relu")(x)
x = Dropout(0.5)(x)

out_age = Dense(10,activation="softmax",name="age_output")(x)
out_gender = Dense(2,activation="softmax",name="gender_output")(x)

model = Model(inputs=inp, outputs=[out_age, out_gender])

model.compile(
    optimizer=Adam(1e-3),
    loss={
        "age_output":"categorical_crossentropy",
        "gender_output":"categorical_crossentropy"
    },
    metrics={
        "age_output":"accuracy",
        "gender_output":"accuracy"
    }
)

model.fit(
    X,
    {"age_output":y_age_oh,"gender_output":y_gender_oh},
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

model.save("age_gender_model.keras")
print("Model saved as age_gender_model.keras")


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 255ms/step - age_output_accuracy: 0.1050 - age_output_loss: 2.4273 - gender_output_accuracy: 0.5475 - gender_output_loss: 0.7817 - loss: 3.2177 - val_age_output_accuracy: 0.1000 - val_age_output_loss: 2.3010 - val_gender_output_accuracy: 0.5800 - val_gender_output_loss: 0.6887 - val_loss: 2.9891
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - age_output_accuracy: 0.0975 - age_output_loss: 2.3008 - gender_output_accuracy: 0.5375 - gender_output_loss: 0.6884 - loss: 2.9892 - val_age_output_accuracy: 0.0700 - val_age_output_loss: 2.3046 - val_gender_output_accuracy: 0.5300 - val_gender_output_loss: 0.6929 - val_loss: 2.9924
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - age_output_accuracy: 0.1075 - age_output_loss: 2.3008 - gender_output_accuracy: 0.5125 - gender_output_loss: 0.6916 - loss: 2.9929 - val_age_output_accuracy: 0.1000 - val_age_output_loss: 2.3007 - val_gender_output_accuracy: 0.5800 - val_gender_output_loss: 0.6897 - val_l